In [63]:
import chess

In [64]:
fen = "3Q4/4K3/8/2k5/8/8/8/8 w - - 0 1"

In [65]:
b = chess.Board(fen)

In [66]:
b.legal_moves

<LegalMoveGenerator at 0x10c9db390 (Qh8, Qg8, Qf8, Qe8, Qc8+, Qb8, Qa8, Qd7, Qc7+, Qd6+, Qb6+, Qd5+, Qa5+, Qd4+, Qd3, Qd2, Qd1, Kf8, Ke8, Kf7, Kd7, Kf6, Ke6)>

In [74]:
list(b.legal_moves)[0]

Move.from_uci('d8h8')

In [67]:
moves = [b.san(m) for m in b.legal_moves]
moves[:5]

['Qh8', 'Qg8', 'Qf8', 'Qe8', 'Qc8+']

In [71]:
m = moves[0]
m

'Qh8'

In [73]:
type(m)

str

In [72]:
dir(m)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'stri

In [62]:
m = b.push(m)

AssertionError: push() expects move to be pseudo-legal, but got d8h8 in 7Q/4K3/8/2k5/8/8/8/8

In [60]:
m

Move.from_uci('d8h8')

In [61]:
b.san(m)

AssertionError: san() and lan() expect move to be legal or null, but got d8h8 in 7Q/4K3/8/2k5/8/8/8/8 b - - 1 1

In [47]:
b.san(b.push_san(m))

AssertionError: san() and lan() expect move to be legal or null, but got d8h8 in 7Q/4K3/8/2k5/8/8/8/8 b - - 1 1

In [26]:
b.legal_moves

<LegalMoveGenerator at 0x10af99510 (Kc6, Kb6, Kd5, Kb5, Kc4, Kb4)>